In [ ]:
# @title 安装环境
import os
import time
from IPython.utils import capture
from IPython.display import clear_output, display, HTML

# 忽略警告
import warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
!git config --global advice.detachedHead false
!sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' {warnings.__file__}
warnings.filterwarnings("ignore")

def check_gpu():
    import tensorflow as tf
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
      print("\033[96m GPU is available.\033[0m", flush=True)
    else:
      print("\033[91m 没有使用GPU,请在代码执行程序-更改运行时类型-设置为GPU!\033[0m", flush=True)
      display(HTML("<img src='https://i.ibb.co/xfb7pB7/check-gpu.png' width='560px'/>"))
      from google.colab import runtime
      runtime.unassign()

%cd /content
#@markdown ###<font color="#11659a"> 选择版本：</font>
_version = "Fooocus" # @param ["Fooocus", "SimpleSDXL"]
#@markdown #####<font color="#2c9678">☑检查是否使用了GPU；☐不检查。
_check_gpu = False  #@param {type:"boolean"}

if _check_gpu:
  check_gpu()
start_time = time.time()
if _version == "Fooocus":
  git_url = "https://github.com/lllyasviel/Fooocus"
elif _version == "RuinedFooocus":
  git_url = "https://github.com/runew0lf/RuinedFooocus"
elif _version == "Fooocus-ControlNet-SDXL":
  git_url = "https://github.com/fenneishi/Fooocus-ControlNet-SDXL"
elif _version == "Fooocus-MRE":
  git_url = "https://github.com/MoonRide303/Fooocus-MRE"
elif _version == "SimpleSDXL":
  git_url = "https://github.com/metercai/SimpleSDXL"

!git clone -q --depth 1  {git_url}
print('项目克隆成功！')
main_dir = f"/content/{_version}"
print("正在安装环境,大约需要两分钟...")

%cd {main_dir}
if os.path.exists("settings-no-refiner.json"):
  !cp settings-no-refiner.json settings.json

with capture.capture_output() as cap:
  !apt-get -qq -y update
  !apt -y install -qq aria2
  !pip install -q pyngrok pycloudflared -U
  !pip uninstall -y lida llmx
  !pip install kaleido cohere openai tiktoken typing-extensions==4.8.0 tensorflow-probability==0.13.0 dopamine-rl
  !pip install -q -r requirements_versions.txt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/metercai/SimpleSDXL/SimpleSDXL/language/cn.json -d {main_dir}/language -o cn.json
  !wget -q https://huggingface.co/Vanwise/sd-colab/resolve/main/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
end_time = time.time()
print("环境安装完成，耗时：{:.3f}分钟.".format((end_time - start_time) / 60))

In [ ]:
%run /content/wise.py
replace_download_models(_version)

In [ ]:

cn = True  #@param {type:"boolean"}
ip = True  #@param {type:"boolean"}
face = True  #@param {type:"boolean"}

default_models = "sai" # @param ["juggernautXL", "realistic", "anime", "sai", ""]
ckpt_link = "" # @param {type:"string"}
loars_link = "" # @param {type:"string"}
emb_link = "" # @param {type:"string"}
_inpaint = "v2.6" # @param ["v1", "v2.5", "v2.6"]

presets = ""
if default_models == "juggernautXL":
    add_juggernautxl(main_dir)
    presets = ""
elif default_models == "realistic":
    add_realistic(main_dir)
    presets = "--preset realistic"
elif default_models == "anime":
    add_anime(main_dir)
    presets = "--preset anime"
elif default_models == "sai":
    add_sai(main_dir)
    presets = "--preset sai"

if ckpt_link and default_models == "":
    set_default_ckpt(main_dir, ckpt_link)

if loars_link and default_models == "":
    set_default_loars(main_dir, loars_link)

add_controlnet(main_dir,cn,ip,face)
add_other(main_dir)
add_vaeapp(main_dir)
add_custom_ckpt(main_dir,ckpt_link)
add_custom_loras(main_dir,loars_link)
add_custom_embeddings(main_dir,emb_link)
add_inpaint(main_dir,_inpaint)

In [ ]:
start_download_models(models)

In [ ]:
# @title (🔆)MOD！
# @markdown #### 💠用于下载主模型、Lora、Vae、Embedding.
import pandas as pd
import requests
from io import BytesIO
from google.colab import files, drive
from concurrent import futures

# @markdown #### 启用MOD:
enable_mod = True #@param {type:"boolean"}
# @markdown #### 模型下载位置:
modown_dir = "colab" # @param ["colab", "drive"]
if modown_dir == "drive":
  modelr_dir = "/content/drive/MyDrive/sd_models"
else:
  modelr_dir = f"{main_dir}/models/"
# @markdown ---
# @markdown ##### [MOD链接](https://docs.google.com/spreadsheets/d/1sp_o8fUBZk8BrThY9w1LeLd2aTuq41sT/edit#gid=149213849):
mod_link = "https://docs.google.com/spreadsheets/d/e/2PACX-1vR_4_XVd-OxQQlCLQo99dnRCi2bBJtjg8Et_gQfgd_-g6j9ltuBAbiwPC6qSJy6_KNi-G_bPZdqrv-y/pub?output=xlsx"# @param {type:"string"}
# @markdown ##### MOD路径:
mod_path = "/content/drive/MyDrive/wise.xlsx"# @param {type:"string"}
# @markdown ##### 代码逻辑:优先使用MOD链接,链接为空使用MOD路径,路径为空使用上传MOD。

def checkmod_link(link):
  response = requests.get(link)
  if response.status_code == 200:
      content = response.content
      try:
          df = pd.read_excel(BytesIO(content))
          print("MOD读取成功!")
          return df
          #print(df.head())
      except Exception as e:
          print("无法正确读取文件,请检查文件格式是否为'.xlsx' :", str(e))
  else:
      print("无法获取文件,请检查链接是否正确!")

def checkmod_path(path):
  try:
    df = pd.read_excel(path)
    print("MOD读取成功!")
    return df
    #print(df.head())
  except Exception as e:
    print("无法正确读取文件,请检查文件格式是否为'.xlsx' :", str(e))

def checkmod_up():
    while True:
        try:
            uploaded = files.upload()
            if not uploaded:
                print("取消上传,使用默认MOD!")
                mod = 'https://github.com/Van-wise/sd-colab/raw/main/wise.xlsx'
                df = pd.read_excel(mod)
                break
            mod_file = os.path.join(os.getcwd(), list(uploaded.keys())[0])
            df = pd.read_excel(mod_file)
            print("MOD读取成功!")
            break  # Exit the loop if the file is successfully read

        except (ValueError, Exception) as e:
            os.remove(mod_file)
            print("无法正确读取文件，请检查文件格式是否为'.xlsx':", str(e))
            print("请重新上传文件。")
    return df

def download_mod(file_dLlink, save_dir, file_name, index):
    try:
        os.system(f"aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {file_dLlink} -d {save_dir} -o {file_name}")
        print(f"第{index+1}行:{file_name}下载成功!")
    except Exception as e:
        print(f"第{index+1}行:{file_name}文件下载错误：", e)

from concurrent import futures

def start_mod():
    if mod_link:
        df = checkmod_link(mod_link)
    elif mod_path:
        df = checkmod_path(mod_path)
    else:
        df = checkmod_up()
    df = df.fillna("")

    with futures.ThreadPoolExecutor() as executor:
        future_to_index = {}

        for index, row in df.iterrows():
            file_switch, file_name, file_type, file_dLlink = row[["开关", "名称", "类型", "下载地址"]]

            if any(value == "" for value in [file_switch, file_name, file_type, file_dLlink]):
                print(f"跳过,第{index+1}行,因为:|{'|'.join([key for key, value in row.items() if value == ''])}|的内容不能为空!")
                continue

            save_dir = os.path.join(modelr_dir, file_type)
            file_path = os.path.join(save_dir, file_name)

            if not os.path.exists(file_path) and file_switch == "on":
                future = executor.submit(download_mod, file_dLlink, save_dir, file_name, index)
                future_to_index[future] = index

        for future in futures.as_completed(future_to_index):
            index = future_to_index[future]
            try:
                result = future.result()
            except Exception as e:
                print(f"第{index+1}行下载错误:", e)
    print("MOD运行完成...🎉")

if enable_mod:
  !apt-get -qq install -y aria2
  start_mod()

In [ ]:
# @title (🔆)MOD！
# @markdown #### 💠用于下载主模型、Lora、Vae、Embedding.
import pandas as pd
import requests
from io import BytesIO
from google.colab import files, drive
from concurrent import futures

# @markdown #### 启用MOD:
enable_mod = True #@param {type:"boolean"}
# @markdown #### 模型下载位置:
modown_dir = "colab" # @param ["colab", "drive"]
if modown_dir == "drive":
  modelr_dir = "/content/drive/MyDrive/sd_models"
else:
  modelr_dir = f"{main_dir}/models/"
# @markdown ---
# @markdown ##### [MOD链接](https://docs.google.com/spreadsheets/d/1o-13mwJiydTprn9OQHia2bWCirCwyLDvt2-OoWT5Trg/edit#gid=149213849):
mod_link = "https://docs.google.com/spreadsheets/d/e/2PACX-1vR_4_XVd-OxQQlCLQo99dnRCi2bBJtjg8Et_gQfgd_-g6j9ltuBAbiwPC6qSJy6_KNi-G_bPZdqrv-y/pub?output=xlsx"# @param {type:"string"}
# @markdown ##### MOD路径:
mod_path = "/content/drive/MyDrive/wise.xlsx"# @param {type:"string"}
# @markdown ##### 代码逻辑:优先使用MOD链接,链接为空使用MOD路径,路径为空使用上传MOD。

if enable_mod:
  !apt-get -qq install -y aria2
  start_mod(mod_link,mod_path,modelr_dir)

In [ ]:
# @title 内网穿透
import os
import shlex
import subprocess
from pathlib import Path
from typing import Union
# @markdown #### [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken):
use_ngrok = True # @param {type:"boolean"}
ngrok_token = "2Wl6oIN6RPlyMnEg25bFjMjUn8n_4bUz2gmvBLc6dSrGZ3Bng" # @param {type:"string"}
# @markdown #### cloudflare:
use_cloudflare = True # @param {type:"boolean"}
# @markdown #### remote:
use_remote = True # @param {type:"boolean"}

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

if use_remote:
  id_rsa_file = "/content/id_rsa"
  id_rsa_pub_file = "/content/id_rsa/id_rsa.pub"
  if os.path.exists(id_rsa_file):
      os.remove(id_rsa_file)
  if os.path.exists(id_rsa_pub_file):
      os.remove(id_rsa_pub_file)
  ssh_name = "id_rsa"
  ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
  gen_key(ssh_path)

if use_ngrok:
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_token)

def start_tunnle():
  time.sleep(5)
  # open("/content/output.log", "w").close()
  # # 循环检测文件内容，直到存在"http://127.0.0.1:9090"。
  # while not "http://127.0.0.1:7865" in open("/content/output.log", "r").read():
  #   time.sleep(1)

  clear_output()
  from IPython.display import display, HTML
  audio_url = "https://github.com/Van-wise/sd-colab/raw/main/qidong.mp3"
  display(HTML(f'<audio src="{audio_url}" controls autoplay style="display:none"></audio>'))

  if use_ngrok:
      try:
          from pyngrok import ngrok
          ngrok_tunnel = ngrok.connect(7865, "http")
          print("ngrok_tunnel:", ngrok_tunnel)
      except Exception as e:
          print("ngrok 连接失败：", e)

  if use_cloudflare:
      try:
          from pycloudflared import try_cloudflare
          cloudflare_url = try_cloudflare(7865, verbose=False)
          print("cloudflare_tunnel:", cloudflare_url)
      except Exception as e:
          print("cloudflare 连接失败：", e)

  if use_remote:
      !ssh -R 80:127.0.0.1:7865 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe 2>&1 | tee -a /content/tunnel.log

In [ ]:
import threading
thread = threading.Thread(target=start_tunnle, daemon=True)
thread.start()

#!python entry_with_update.py --language zh --port 7685
!python launch.py --language cn --port 7865 $presets